In [2]:
import gspread
import pandas as pd

gc = gspread.service_account(filename="./crudg.json")
sheet = gc.open("flightsched").get_worksheet(3)
dataframe = pd.DataFrame(sheet.get_all_records())
dataframe.head(2)

,A/C No,Flt Desg,Freq,Dept Arp,Dept Time,Arvl Arp,Arrv Time,Equip,Block Time,Eff Date,Dis Date
0,1,149,6,HYD,10:10,IXB,12:30,321,2:20,1-May-21,1-May-21
1,1,619,6,IXB,13:05,HYD,15:20,321,2:15,1-May-21,1-May-21


In [3]:
%%time
%load_ext line_profiler

Wall time: 185 ms


- [x] memory usage: 4.5+ MB for month of April needs to be  reduced 
- [x] action data types specified reduced space to 2.3 mb 
- [ ] there are negative transit time , same needs to be manually confirmed if they are being caused by source error or code error 
- [ ] what happens to daterange when start date and end date are same 
- [ ] import opininated code formatter black 
- [ ] trying to solve negative transit times need to check percentage of negative transit times 

In [4]:
import IPython

# the below code ensures multiple outputs are then in lab cells
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [27]:
%%time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread

import datetime
import pickle
import re
from altair.vegalite.v4.schema.channels import Tooltip
import altair as alt
import networkx as nx
import itertools
import math
import random
from collections import defaultdict

Wall time: 0 ns


In [6]:
# altair disable max rows
alt.data_transformers.disable_max_rows()
# altair enble chart
alt.renderers.enable("mimetype")

DataTransformerRegistry.enable('default')

RendererRegistry.enable('mimetype')

In [7]:
# Set ipython's max row display
pd.set_option("display.max_row", 50)

# Set iPython's max column width to 50
pd.set_option("display.max_columns", 50)

In [8]:
alpha = pickle.load(open("apr_21.p", "rb"))
# alpha.info()

In [9]:
alpha["cal_date"][0]
reqDf = alpha.loc[
    (alpha["Dept_Arp"] == "VTZ") & (alpha["cal_date"] == alpha["cal_date"][0])
]
reqDf = reqDf.sort_values(by=["engg_dep"])
reqDf = reqDf.loc[reqDf["transit_time"] > pd.to_timedelta("15.5us")]
reqDf = reqDf[
    [
        "Dept_Arp",
        "engg_arr",
        "engg_dep",
        "transit_time",
        "Flt_Desg",
        "Text_Comment",
        "Subfleet",
        "Arvl_Arp",
        "aocs_arr",
        "deptype",
        "Block_Time",
        "Service_Type",
        "cal_date",
    ]
]
reqDf = reqDf.reset_index(drop=True)
reqDf

Timestamp('2021-04-01 00:00:00')

,Dept_Arp,engg_arr,engg_dep,transit_time,Flt_Desg,Text_Comment,Subfleet,Arvl_Arp,aocs_arr,deptype,Block_Time,Service_Type,cal_date
0,VTZ,2021-04-01 07:05:00,2021-04-01 07:35:00,0 days 00:30:00,6E 0647,A170,6E 32S,DEL,2021-04-01 10:00:00,Transit,0 days 02:25:00,J,2021-04-01
1,VTZ,2021-04-01 07:35:00,2021-04-01 08:05:00,0 days 00:30:00,6E 0608,A141,6E 32S,HYD,2021-04-01 09:15:00,Transit,0 days 01:10:00,J,2021-04-01
2,VTZ,2021-04-01 08:10:00,2021-04-01 08:40:00,0 days 00:30:00,6E 0803,A321_05,6E 321,MAA,2021-04-01 10:10:00,Transit,0 days 01:30:00,J,2021-04-01
3,VTZ,2021-04-01 08:20:00,2021-04-01 08:50:00,0 days 00:30:00,6E 0133,A130,6E 32S,BLR,2021-04-01 10:30:00,Transit,0 days 01:40:00,J,2021-04-01
4,VTZ,2021-04-01 09:45:00,2021-04-01 10:15:00,0 days 00:30:00,6E 6038,A323_019,6E 320,CCU,2021-04-01 11:50:00,Transit,0 days 01:35:00,J,2021-04-01
5,VTZ,2021-04-01 12:25:00,2021-04-01 13:00:00,0 days 00:35:00,6E 6809,A092,6E 32S,HYD,2021-04-01 14:15:00,Transit,0 days 01:15:00,J,2021-04-01
6,VTZ,2021-04-01 14:05:00,2021-04-01 14:30:00,0 days 00:25:00,6E 6001,A040,6E 32S,MAA,2021-04-01 15:45:00,Transit,0 days 01:15:00,J,2021-04-01
7,VTZ,2021-04-01 16:10:00,2021-04-01 16:40:00,0 days 00:30:00,6E 0783,A092,6E 32S,HYD,2021-04-01 18:00:00,Transit,0 days 01:20:00,J,2021-04-01
8,VTZ,2021-04-01 17:00:00,2021-04-01 17:20:00,0 days 00:20:00,6E 7129,ATR051,6E ATR,VGA,2021-04-01 18:25:00,Transit,0 days 01:05:00,J,2021-04-01
9,VTZ,2021-04-01 17:30:00,2021-04-01 18:40:00,0 days 01:10:00,6E 2772,A058,6E 32S,DEL,2021-04-01 21:05:00,Transit,0 days 02:25:00,J,2021-04-01


In [10]:
jNodeList = reqDf.reset_index(drop=True).to_dict("index").items()
# next(jNodeList)


In [11]:
G = nx.DiGraph()
G.add_nodes_from(jNodeList)

In [12]:
reqList = list(itertools.permutations(G.nodes(data=True), 2))

filtList = list(filter(lambda x: x[0][1]["engg_dep"] < x[1][1]["engg_arr"], reqList))

finalEdgeList = [((x[0][0]), x[1][0]) for x in filtList]
# finalEdgeList

In [13]:
G.add_edges_from(finalEdgeList)

In [14]:
roots = [v for v, d in G.in_degree() if d == 0]
tips = [v for v, d in G.out_degree() if d == 0]
topsortedList = list(nx.algorithms.dag.topological_sort(G))
roots
tips
topsortedList
G.nodes[9]["aocs_arr"]

[0, 1]

[12]

[1, 0, 3, 2, 4, 5, 6, 7, 8, 11, 10, 9, 12]

Timestamp('2021-04-01 21:05:00')

In [15]:
list(G.neighbors(8))
list(G.adj[8].items())
list(G.successors(8))
sorted(G.neighbors(12), key=lambda x: G.nodes[x]["engg_arr"])
list(G.neighbors(12))

[9, 10, 11, 12]

[(9, {}), (10, {}), (11, {}), (12, {})]

[9, 10, 11, 12]

[]

[]

- [ ]  check if all nodes have been covered else 
- [ ] edge case One station vtz date first april aloocation of fights between 1700 and 1900 theoritical required manpower 3  actual required manpower 2 

In [16]:
%%black
def DfsUtil(startvertex, visited, paramPath):

    visited["visitStatus"]  = True
    paramPath.append(startvertex)
    if list(G.neighbors(startvertex)) == []:
        return

        

    nextToVisit = sorted(G.neighbors(startvertex), key=lambda x: G.nodes[x]["engg_arr"])
    nextToVisit = [xray for xray in nextToVisit if  visited[xray][0]]
    # print(nextToVisit)
    if nextToVisit == []:
        return

    nextToVisit = nextToVisit[0]
    if nextToVisit not in visited:
        DfsUtil(nextToVisit, visited, paramPath)


def DFS(Graph):
    roots = [v for v, d in G.in_degree() if d == 0]
    def def_value():
        return {"visitStatus" :False,"visitPath":[]}
    rootpathDict = defaultdict(def_value)
    flagAllNodevisited = set(G.nodes()).difference(set(sorted({x for v in testOne.values() for x in v})))) == set

    # first pass via roots 
    for xray in roots:
        graphPath = []
        DfsUtil(xray, rootpathDict, graphPath)
        rootpathDict["visitPath"] = graphPath
    # second pass via nodes left after first path 
    if  :
        return rootpathDict
    else :




    


testOne = DFS(G)
testOne

{0: [0, 2, 4, 5, 6, 7, 8, 9, 12], 1: [1, 3, 10]}

In [37]:

set(G.nodes()).difference(set(sorted({x for v in testOne.values() for x in v})))
def def_value():
        return {"visitStatus" :False,"visitPath":[]}
rootpathDict = defaultdict(def_value)







{11}

True

In [28]:
list(G.nodes(data=False))

d["a"]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

[False, []]

- checking for negative times 

In [ ]:
negAlpha = alpha.loc[
    ~(alpha["transit_time"] > pd.to_timedelta("15.5us"))
    & (alpha["deptype"] == "Transit")
]
# negAlpha["Text_Comment"].unique().tolist()

In [ ]:
negAlpha.head(5)
testdate = negAlpha["cal_date"][184]

In [ ]:
x_df = alpha.loc[(alpha["Text_Comment"] == "A001") & (alpha["cal_date"] == testdate)]
x_df

- [ ] when bolean masking transit time > 0 micro seconds also removes the NAt also error 

In [ ]:
groupedDf = alpha.copy()
groupedDf = (
    groupedDf.groupby(["cal_date", "deptype", "Dept_Arp"])
    .size()
    .reset_index(name="count")
)

groupedDf["count"] = pd.to_numeric(groupedDf["count"])
groupedDf.info()

In [ ]:
pd.Timedelta("30 min")

In [ ]:
df = groupedDf.loc[groupedDf["deptype"] == "Transit"].copy()

In [ ]:
listAirports = df["Dept_Arp"].unique().tolist()
randomlistAirports = random.sample(listAirports, 5)
randomlistAirports
sortedDf = df.loc[df["Dept_Arp"].isin(randomlistAirports)]
sortedDf

In [ ]:
def getMonthChart(
    dataframe: pd.DataFrame, stationList: list = None
) -> alt.vegalite.v4.api.VConcatChart:
    """[summary]

    Args:
        month ([type]): [description]
        station ([type]): [description]

    Returns:
        [type]: [description]
    TODO : chart height claculation based on
    """
    # checking Area
    # test-One ensuring the type of keyword parameter and throwing error
    if stationList != None:
        assert isinstance(
            stationList, list
        ), "stationList should be a (pythonType(List))"
    # ensuring argument is of type pandas datafrme
    assert isinstance(
        dataframe, pd.DataFrame
    ), "dataframe should be of type #pandas Dataframe #"

    groupedDf = dataframe.copy()
    groupedDf = (
        groupedDf.groupby(["cal_date", "deptype", "Dept_Arp"])
        .size()
        .reset_index(name="count")
    )

    if stationList != None:
        # test three if stationList is subset of avilable stations
        assert set(stationList).issubset(
            set(dataframe["Dept_Arp"].unique())
        ), "statonlist station not in dataframe['Dept_Arp']"

        groupedDf = groupedDf.loc[groupedDf["Dept_Arp"].isin(stationList)]

    listOperationsType = groupedDf["deptype"].unique().tolist()
    # claulating chart size for chart Height

    chartHeight = math.ceil(len(groupedDf["Dept_Arp"].unique()) / 5) * 100

    # base Chart
    base = alt.Chart().encode(
        alt.X(
            "monthdate(cal_date):O",
            title="Days Of The Month",
            scale=alt.Scale(paddingInner=0),
        ),
        alt.Y("Dept_Arp:N", title="Airport", scale=alt.Scale(paddingInner=0)),
    )

    # rect chart
    alpha = base.mark_rect().encode(
        color=alt.Color("count:Q", legend=alt.Legend(direction="horizontal")),
        tooltip=[
            alt.Tooltip("cal_date", title="Date"),
            alt.Tooltip("count", title="No-of-Ops"),
            alt.Tooltip("deptype", title="Type-Ops"),
            alt.Tooltip("Dept_Arp", title="Aiport"),
        ],
    )

    # text chart
    beta = base.mark_text(align="center", baseline="middle").encode(text="count:Q")

    # chart with data as one of Transit . base Dep , base base Arr
    chartOne = (
        alt.layer(
            alpha,
            beta,
            data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[0]].copy(),
        )
        .properties(
            title=str(listOperationsType[0]) + " Ops in Station",
            width=1000,
            height=chartHeight,
        )
        .interactive()
    )
    # chart with data as one of Transit . base Dep , base base Arr
    chartTwo = (
        alt.layer(
            alpha,
            beta,
            data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[1]].copy(),
        )
        .properties(
            title=str(listOperationsType[1]) + " Ops in Station",
            width=1000,
            height=chartHeight,
        )
        .interactive()
    )
    # chart with data as one of Transit . base Dep , base base Arr
    chartThree = (
        alt.layer(
            alpha,
            beta,
            data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[2]].copy(),
        )
        .properties(
            title=str(listOperationsType[2]) + " Ops in Station",
            width=1000,
            height=chartHeight,
        )
        .interactive()
    )
    return alt.vconcat(chartOne, chartTwo, chartThree)

In [ ]:
finalChart = getMonthChart(alpha, stationList=randomlistAirports)

In [ ]:
finalChart

In [ ]:
#  datetime.datetime.strptime("Apr-21", "%b-%y")
pd.to_datetime("Apr-21", format="%b-%y").daysinmonth